In [1]:
import urllib

In [2]:
from bs4 import BeautifulSoup as soup

In [3]:
import pandas as pd

In [4]:

def read_link(link):
    f = urllib.request.urlopen(link)
    myfile = f.read()
    soup_file = soup(myfile, "html")
    return soup_file

In [5]:
def get_game_log_soup(link):
    soup_file = read_link(link)

    game_log_suffix = soup_file.findAll("table", 
                                        {"class",
                                         "data borderTop"})[0].findAll(
        "a",href = True)[0]['href']

    game_log_link = website + game_log_suffix
    game_log_soup = read_link(game_log_link)

    return game_log_soup

In [6]:

def get_game_log_data(game_log_soup, player_name):

    game_log_data = []
    for row in ['row1', 'row2']:

        row_data = game_log_soup.findAll("table", 
                              {"class": 
                               "data borderTop title"}
                             )[0].findAll("tr", {"class":row})

        for dat in row_data:
            gl = [player_name] + dat.get_text(",").split(",")
            game_log_data.append(gl)

    res = pd.DataFrame(game_log_data)
    return res

In [7]:
def get_soup_file(start_row):
    
    link = "https://www.cbssports.com/nba/stats/playersort/NBA/year-2017-season-regularseason-category-scoringpergame?start_row=" + str(start_row)

    f = urllib.request.urlopen(link)
    myfile = f.read()
    soup_file = soup(myfile, "html")

    return soup_file

In [8]:
def get_clean_data(soup_file):


    data = soup_file.findAll("table", {"class", "data"})[0]

    clean_data = []

    for r in ['row1', 'row2']:

        row_df = data.findAll("tr", {"class", r})

        for i in range(len(row_df)):
            clean_data.append(row_df[i].get_text(",").split(","))
            
    return pd.DataFrame(clean_data)

In [9]:

scoring_per_game = []

for i in range(17):
    start_row = i*30 + 1
    scoring_per_game.append(get_clean_data(get_soup_file(start_row)))

/Users/davinkaing/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/davinkaing/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [10]:
scoring_per_game_df = pd.concat(scoring_per_game)

In [11]:
scoring_per_game_df.to_csv("scoring_per_game.csv", index = False)

In [24]:

all_game_log_data = []

for i in range(17):
    start_row = i*30 + 1

    link = "https://www.cbssports.com/nba/stats/playersort/NBA/year-2017-season-regularseason-category-scoringpergame?start_row=" + str(start_row)

    f = urllib.request.urlopen(link)
    myfile = f.read()
    soup_file = soup(myfile, "html")

    table = soup_file.findAll("table", {"class", "data"})[0]

    href_dat = table.findAll("td", {"align":"left"})


    for j in range(len(href_dat)):

        website = "https://www.cbssports.com/"
        ph = href_dat[j].findAll("a", href = True)[0]
        player_name = ph.get_text().replace(" ", "-").replace(".", "-")
        player_href = ph['href'] + player_name
        player_link = website + player_href
        game_log_soup = get_game_log_soup(player_link)
        all_game_log_data.append(get_game_log_data(game_log_soup, 
                                                   player_name))


/Users/davinkaing/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/davinkaing/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [25]:
all_game_log_df = pd.concat(all_game_log_data, axis = 0)

In [26]:
all_game_log_df.to_csv("all_game_log.csv", index = False)

In [29]:
len(all_game_log_df[0].unique())

500